In [ ]:
# Jupyter Notebook Cell 1: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter

# For nicer plots in Jupyter
%matplotlib inline

# ----------------------------------------------------
# 1. Simulate Example Data
# ----------------------------------------------------
np.random.seed(42)
N = 200
df = pd.DataFrame({
    'time': np.random.exponential(24, size=N).clip(0,60),  # ~ up to 60 months
    'event': np.random.binomial(1, p=0.4, size=N),         # ~40% events
    'chole': np.random.binomial(1, p=0.25, size=N),        # 25% had chole
    'blood_loss': np.random.randint(0, 2, size=N),         # 0 or 1
    'age': np.random.normal(60, 10, size=N),               # ~60 ± 10
    'sex': np.random.binomial(1, p=0.5, size=N)            # 1=male, 0=female
})
df.head()

# ----------------------------------------------------
# 2. Fit a Multivariable Cox Model
# ----------------------------------------------------
cph = CoxPHFitter()
# Assume the outcome is "time to event" with 'event'=1 if recurrence/death occurred
cph.fit(
    df,
    duration_col='time',
    event_col='event',
    formula='chole + blood_loss + age + sex'
)

# Print model summary: hazard ratios, p-values, etc.
cph.print_summary()

# ----------------------------------------------------
# 3. Plot Adjusted Survival Curves by Key Factor
#    Using lifelines' built-in partial effects method
# ----------------------------------------------------
fig, ax = plt.subplots(figsize=(7,5))

cph.plot_partial_effects_on_outcome(
    covariates = [
        {'chole': 0, 'blood_loss': df['blood_loss'].mean(),
         'age': df['age'].mean(), 'sex': df['sex'].mean()},
        {'chole': 1, 'blood_loss': df['blood_loss'].mean(),
         'age': df['age'].mean(), 'sex': df['sex'].mean()}
    ],
    values = [0, 1],
    ax = ax
)

ax.set_title("Adjusted Survival Curves by Cholecystectomy Status", fontsize=14, fontweight='bold')
ax.set_xlabel("Time (months)", fontsize=12)
ax.set_ylabel("Survival Probability", fontsize=12)
ax.legend(["Chole=0 (No History)", "Chole=1 (History)"], fontsize=10)
ax.grid(True, alpha=0.3)
plt.show()

# ----------------------------------------------------
# 4. (Optional) Plot Adjusted Cumulative Incidence
#    (1 - survival) for the same scenarios
# ----------------------------------------------------
fig, ax2 = plt.subplots(figsize=(7,5))

partial_df = cph.predict_survival_function(
    X = pd.DataFrame({
        'chole': [0,1],
        'blood_loss': [df['blood_loss'].mean()]*2,
        'age': [df['age'].mean()]*2,
        'sex': [df['sex'].mean()]*2
    })
)

# partial_df's index ~ time points, columns = 0 or 1 for chole
cum_inc = 1 - partial_df

for i, col_name in enumerate(cum_inc.columns):
    ax2.plot(cum_inc.index, cum_inc[col_name], label=f"Chole={col_name}")

ax2.set_title("Adjusted Cumulative Incidence by Cholecystectomy Status", fontsize=14, fontweight='bold')
ax2.set_xlabel("Time (months)", fontsize=12)
ax2.set_ylabel("Cumulative Incidence", fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
plt.show()
